In [3]:
import json

task = ['public']
data = {}

for t in task:
    with open(f'../data/{t}.jsonl', 'r') as json_file:
        json_list = list(json_file)
        data[t] = [json.loads(json_str) for json_str in json_list]
        print(len(data[t]))

5494


## Preprocessing the data

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

2022-05-15 22:54:06.682893: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-15 22:54:06.682919: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
import os
import torch
from tw_rouge.tw_rouge import get_rouge

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} for training...')

model.to(device)

/home/ai2lab/anaconda3/envs/nlp/lib/python3.9/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/home/ai2lab/anaconda3/envs/nlp/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/home/ai2lab/anaconda3/envs/nlp/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
2022-05-15 22:54:21.222175: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to c

Using cuda for training...


/home/ai2lab/anaconda3/envs/nlp/lib/python3.9/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (w

In [17]:
import torch
model.load_state_dict(torch.load('../weight/checkpoint-20000.pt'))

<All keys matched successfully>

In [18]:
from torch.utils.data import Dataset, DataLoader

max_length = 384

def preprocess(articles):
    encode_articles = {}

    maintext = [article["maintext"] for article in articles]
    
    # Tokenize
    encode_articles = tokenizer(maintext, 
                                max_length=max_length,
                                truncation=True, 
                                padding=True)
    
    if 'title' in articles[0].keys():
        titles = [article["title"] for article in articles]
        encode_articles['title'] = tokenizer(titles, 
                                            max_length=56,
                                            truncation=False, 
                                            padding=True)
    return encode_articles

In [19]:
encoded_datasets = {}
encoded_datasets = {t: preprocess(data[t]) for t in task}

/home/ai2lab/anaconda3/envs/nlp/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class Summary(Dataset):
    def __init__(self, encoded_dataset):
        self.encode_token = encoded_dataset["input_ids"]
        self.label = encoded_dataset["title"]['input_ids'] if "title" in encoded_dataset.keys() else None
        
    def __getitem__(self, index):
        if self.label is None:
            return torch.tensor(self.encode_token[index])
        else:
            return torch.tensor(self.encode_token[index]), torch.tensor(self.label[index])

    def __len__(self):
        return len(self.encode_token)

In [21]:
valid = Summary(encoded_datasets['public'])

BATCH_SIZE = 128

validloader = DataLoader(dataset=valid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

## Fine-tuning the model

In [22]:
from tqdm import tqdm

def compute_metrics(pred, label):
    decode_pred = [tokenizer.decode(p, skip_special_tokens=True) for p in pred]
    decode_label = [tokenizer.decode(l, skip_special_tokens=True) for l in label]
    score = get_rouge(decode_pred, decode_label, ignore_empty=True)
    print(f'{score}')
    return score

model.eval()
label_list, eval_predictions = [],[]

k=5
print(f'K = {k}')
with torch.no_grad():
    for input_ids, labels in tqdm(validloader):
        input_ids, labels = input_ids.to(device), labels.to(device)
        outputs = model.generate(input_ids, num_beams=k, max_length=64)
        eval_predictions += outputs
        label_list+=labels
    compute_metrics(eval_predictions, label_list)


K = 5


 93%|█████████▎| 40/43 [04:38<00:20,  6.93s/it]

In [ ]:
decode_pred = [tokenizer.decode(p, skip_special_tokens=True) for p in eval_predictions]

In [ ]:
ids = [d['id'] for d in data['public']]

In [ ]:
with open('pred.jsonl', 'w') as f:
    for t, i in zip(decode_pred, ids):
        f.write("{"+f'"title": "{t}", "id": "{i}"'+"}\n")

In [ ]:
model.save_pretrained('../best_weight')
tokenizer.save_pretrained('../best_weight')